# Calling CN.listObjects

This script demonstrates interacting with the [CN.listObjects](https://purl.dataone.org/architecture/apis/CN_APIs.html#CNRead.listObjects) method using the python client.

In [46]:
# Import the library and create a client instance

from d1_client import baseclient_2_0

cn_base_url = "https://cn.dataone.org/cn"
client = baseclient_2_0.DataONEBaseClient_2_0(cn_base_url)

Specify that only five (5) results are to be returned in the request, and start from the first entry. Then call the `listObjects` method.

In [47]:
num_to_retrieve = 5
starting_index = 0
params = {'count': num_to_retrieve,
          'start': starting_index
         }
response = client.listObjects( **params )

Show the response, printing out each entry.

In [48]:
DATE_FORMAT = "%Y-%m-%dT%H:%M:%SZ"
from datetime import datetime as dt

def printResults(response):
    print("Total objects: {0} Start: {1}  Page size: {2}\n".format(response.total, response.start, response.count))
    counter = response.start
    for entry in response.objectInfo:
        print(u"{:08d}: ".format(counter))
        print(u"            PID: {0}".format(entry.identifier.value()))
        print(u"       formatId: {0}".format(entry.formatId))
        print(u"           size: {0}".format(entry.size))
        print(u"  date_modified: {0}".format(entry.dateSysMetadataModified.strftime(DATE_FORMAT)))
        print("")
        counter += 1

printResults(response)

Total objects: 1773489 Start: 0  Page size: 5

00000000: 
            PID: 00010115-8c1f-4813-abc0-f63acf23111d
       formatId: FGDC-STD-001.1-1999
           size: 15988
  date_modified: 2016-10-29T04:05:24Z

00000001: 
            PID: {00030692-0FE1-4A1B-955E-A2E55D659267}
       formatId: http://www.isotc211.org/2005/gmd-noaa
           size: 54120
  date_modified: 2016-03-24T21:28:55Z

00000002: 
            PID: {00042F80-06A6-4C5D-A4F4-4D78E7DC51D9}
       formatId: http://www.isotc211.org/2005/gmd-noaa
           size: 176191
  date_modified: 2016-03-24T22:10:03Z

00000003: 
            PID: 000440bc-c10b-46c7-aaa5-63794f1a9eac
       formatId: FGDC-STD-001-1998
           size: 9573
  date_modified: 2014-06-04T01:10:36Z

00000004: 
            PID: {0004562E-367F-408A-8126-7EB25E525809}
       formatId: http://www.isotc211.org/2005/gmd-noaa
           size: 46124
  date_modified: 2016-05-27T07:04:16Z



Add a `fromDate` parameter, so `listObjects` will respond with the list of entries that were modified between one day ago and now.

In [49]:
import dateparser

start_date = dateparser.parse('yesterday UTC', 
                              settings={'RETURN_AS_TIMEZONE_AWARE': True})

params = {'count': num_to_retrieve,
          'start': starting_index,
          'fromDate': start_date,
         }
print( str(params) )
response = client.listObjects( **params )

printResults( response )


{'count': 5, 'start': 0, 'fromDate': datetime.datetime(2017, 4, 26, 20, 50, 32, 646736, tzinfo=<StaticTzInfo 'UTC'>)}
Total objects: 1424 Start: 0  Page size: 5

00000000: 
            PID: 0209637c-e702-4423-83ec-6642d439c3eb
       formatId: FGDC-STD-001.1-1999
           size: 20889
  date_modified: 2017-04-27T05:05:16Z

00000001: 
            PID: 02c96d88-5a56-41d5-9f47-0c2f63af7c34
       formatId: FGDC-STD-001.1-1999
           size: 33990
  date_modified: 2017-04-27T05:05:09Z

00000002: 
            PID: 03474bf8-64c8-4a9f-a462-fec010cd9c97
       formatId: FGDC-STD-001.1-1999
           size: 11853
  date_modified: 2017-04-27T05:05:04Z

00000003: 
            PID: 06ed00fe-5f8f-4035-bcd4-146bf78b9d87
       formatId: FGDC-STD-001.1-1999
           size: 11110
  date_modified: 2017-04-27T05:05:03Z

00000004: 
            PID: 0db1baa6-3eee-4ed0-a7c7-55465a853e4b
       formatId: FGDC-STD-001.1-1999
           size: 14051
  date_modified: 2017-04-27T05:05:39Z



The server will limit the total number of records returned. When requesting large sets of entries, the 
response will need to be examined to determine if additional pages of results should be requested.

In [52]:
start_date = dateparser.parse('two weeks ago UTC', 
                              settings={'RETURN_AS_TIMEZONE_AWARE': True})
end_date = dateparser.parse('one week ago UTC', 
                              settings={'RETURN_AS_TIMEZONE_AWARE': True})
max_to_retrieve = 25  # limit total numbe of entries to download

params = {'start': 0,  
          'count': 3,      #specify a small page size
          'fromDate': start_date,
          'toDate': end_date}
response = client.listObjects( **params )

if max_to_retrieve > response.total:
    max_to_retrieve = response.total

printResults( response )

num_retrieved = response.count
while num_retrieved < max_to_retrieve:
    params['start'] += response.count
    response = client.listObjects( **params )
    num_retrieved += response.count
    printResults( response )
    

Total objects: 1608 Start: 0  Page size: 3

00000000: 
            PID: {0008F1ED-7499-48E1-B5CE-18359542659F}
       formatId: http://www.isotc211.org/2005/gmd-noaa
           size: 104628
  date_modified: 2017-04-14T07:03:03Z

00000001: 
            PID: {133E1F0D-9F6D-418A-87C7-6EA2C100A46C}
       formatId: http://www.isotc211.org/2005/gmd-noaa
           size: 78461
  date_modified: 2017-04-14T07:03:05Z

00000002: 
            PID: {25D44414-1AFD-4CAE-AA6E-60EC21AF9DDA}
       formatId: http://www.isotc211.org/2005/gmd-noaa
           size: 80441
  date_modified: 2017-04-14T07:03:28Z

Total objects: 1608 Start: 3  Page size: 3

00000003: 
            PID: {41105B7E-B83D-4F54-B610-6735B1073F89}
       formatId: http://www.isotc211.org/2005/gmd-noaa
           size: 82516
  date_modified: 2017-04-14T07:03:26Z

00000004: 
            PID: {48A23084-A3D9-44CA-8A29-E5922F307EE7}
       formatId: http://www.isotc211.org/2005/gmd-noaa
           size: 77357
  date_modified: 2017-04-14T07